In [1]:
import os
import pandas as pd

# change current working directory
default_dir = '..'

if os.getcwd() != default_dir:
    os.chdir(default_dir)

data = pd.read_parquet("..")

pd.set_option('display.max_columns', None)
# data.head(10)

# research/models/AAM_model

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/home/u839129/'

In [2]:
# drop vars with missing values. will replace with new columns
na = data.columns[data.isna().any()].tolist()
print(na)

['TEMP', 'TEMP_I', 'HRTRT', 'HRTRT_I', 'RSPRT', 'RSPRT_I', 'RSPRT_W', 'BPSYS', 'BPDIA', 'SAT', 'SAT_I', 'SAT_W', 'AVPU', 'SHOCKINDEX', 'ANIONGAP', 'BICARB', 'GLUCOSE', 'HEMAT', 'LACT', 'BUN', 'CREAT', 'TROP', 'PML', 'BIL', 'ALB', 'PAC', 'PAO', 'PH']


In [3]:
data = data.drop(columns=na)

In [4]:
# replace by renaming the new columns
for col in [c for c in data.columns if c.endswith('_1')]:
    origin_col = col[:-2]
    data[origin_col] = data[col]

cols_to_drop = [c for c in data.columns if c.endswith('_1')]
data.drop(columns=cols_to_drop, inplace=True)

list(data.columns)

['index',
 'DateTime',
 'AdmissionID',
 'PatientID',
 'ELOS',
 'AGE',
 'SEX',
 'ADMIT1',
 'ADMIT2',
 'ADMIT3',
 'ADMIT4',
 'SEASON1',
 'SEASON2',
 'SEASON3',
 'DAYTIME1',
 'DAYTIME2',
 'DAYTIME3',
 'DAYTIME4',
 'CARE_ORDERDNR',
 'CARE_ORDERFULL_CODE',
 'CARE_ORDERPARTIAL',
 'BPSYS_I',
 'SODIUM',
 'WBC',
 'MISS_TROP',
 'LAPS2',
 'LAPS2_HET',
 'ELOSLAPS2',
 'OXY_SUP',
 'I_OR_ACUTE',
 'I_OR_ELECTIVE',
 'I_OR_UNKNOWN',
 'I_IC',
 'I_EVENT',
 'I_EVENT_V2',
 'I_PRE',
 'I_PRE_V2',
 'EID',
 'AAM',
 'ANIONGAP',
 'BICARB',
 'BPDIA',
 'BPSYS',
 'GLUCOSE',
 'HEMAT',
 'HRTRT',
 'LACT',
 'LBUN',
 'LCREAT',
 'LELOS',
 'LELOSLAPS2',
 'LHRTRT_I',
 'LSAT_I',
 'LOGIT_AGE',
 'LOGIT_SAT',
 'LOGIT_SAT_W',
 'RSPRT_I',
 'LTEMP_I',
 'PML',
 'RSPRT',
 'RSPRT_W',
 'SHOCK',
 'TEMP',
 'TROP']

In [5]:
# rename the vars start with L or LOGIT_
excl = ['LAPS2', 'LAPS2_HET']

renamed = {}

for col in data.columns:
    if col in excl:
        continue
    if col.startswith('LOGIT_'):
        new_name = col[len('LOGIT_'):]
        renamed[col] = new_name
    elif col.startswith('L'):
        new_name = col[1:]
        renamed[col] = new_name

for old, new in renamed.items():
    data[new] = data[old]

data.drop(columns=list(renamed.keys()), inplace=True)

print(list(data.columns))

['index', 'DateTime', 'AdmissionID', 'PatientID', 'ELOS', 'AGE', 'SEX', 'ADMIT1', 'ADMIT2', 'ADMIT3', 'ADMIT4', 'SEASON1', 'SEASON2', 'SEASON3', 'DAYTIME1', 'DAYTIME2', 'DAYTIME3', 'DAYTIME4', 'CARE_ORDERDNR', 'CARE_ORDERFULL_CODE', 'CARE_ORDERPARTIAL', 'BPSYS_I', 'SODIUM', 'WBC', 'MISS_TROP', 'LAPS2', 'LAPS2_HET', 'ELOSLAPS2', 'OXY_SUP', 'I_OR_ACUTE', 'I_OR_ELECTIVE', 'I_OR_UNKNOWN', 'I_IC', 'I_EVENT', 'I_EVENT_V2', 'I_PRE', 'I_PRE_V2', 'EID', 'AAM', 'ANIONGAP', 'BICARB', 'BPDIA', 'BPSYS', 'GLUCOSE', 'HEMAT', 'HRTRT', 'RSPRT_I', 'PML', 'RSPRT', 'RSPRT_W', 'SHOCK', 'TEMP', 'TROP', 'ACT', 'BUN', 'CREAT', 'HRTRT_I', 'SAT_I', 'SAT', 'SAT_W', 'TEMP_I']


In [6]:
# OXY_SUP just being boolean value

In [7]:
# drop unnecessary columns for training/target
cols_to_drop = [
# 'LAPS2_HET',
    
# # input so don't drop
# 'LAPS2',
# 'ELOSLAPS2',
# 'ELOS'

'I_EVENT',
'I_EVENT_V2',
'index',
    
# # can drop this before running emm
# 'DateTime',
# 'AdmissionID',
# 'PatientID',
# 'EID'
]
data = data.drop(columns=cols_to_drop)

In [9]:
pd.options.display.float_format = '{:.2f}'.format
data['AAM'].value_counts().sort_index()

AAM
0.00     1
0.00     1
0.00     1
0.00     1
0.00     1
        ..
99.82    1
99.82    1
99.86    1
99.91    1
99.96    1
Name: count, Length: 9649585, dtype: int64

In [10]:
data['AAM'].min()

2.633271442606753e-12

In [11]:
# # as the emm is fit into the survival probabilty, we use inverse of aam (kind of risk score)
# # or -data['AAM']
# data['AAM_INVERSE'] = -data['AAM']
# data = data.drop(columns=['AAM'])

In [12]:
# data['AAM_INVERSE'].value_counts().sort_index()

In [13]:
data.to_parquet("escobar-working-area/Dayeong/data_aam/data_aam_cleaned.parquet")

In [14]:
list(data.columns)

['DateTime',
 'AdmissionID',
 'PatientID',
 'ELOS',
 'AGE',
 'SEX',
 'ADMIT1',
 'ADMIT2',
 'ADMIT3',
 'ADMIT4',
 'SEASON1',
 'SEASON2',
 'SEASON3',
 'DAYTIME1',
 'DAYTIME2',
 'DAYTIME3',
 'DAYTIME4',
 'CARE_ORDERDNR',
 'CARE_ORDERFULL_CODE',
 'CARE_ORDERPARTIAL',
 'BPSYS_I',
 'SODIUM',
 'WBC',
 'MISS_TROP',
 'LAPS2',
 'LAPS2_HET',
 'ELOSLAPS2',
 'OXY_SUP',
 'I_OR_ACUTE',
 'I_OR_ELECTIVE',
 'I_OR_UNKNOWN',
 'I_IC',
 'I_PRE',
 'I_PRE_V2',
 'EID',
 'AAM',
 'ANIONGAP',
 'BICARB',
 'BPDIA',
 'BPSYS',
 'GLUCOSE',
 'HEMAT',
 'HRTRT',
 'RSPRT_I',
 'PML',
 'RSPRT',
 'RSPRT_W',
 'SHOCK',
 'TEMP',
 'TROP',
 'ACT',
 'BUN',
 'CREAT',
 'HRTRT_I',
 'SAT_I',
 'SAT',
 'SAT_W',
 'TEMP_I']